In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from scipy.stats import gaussian_kde
from plotly.subplots import make_subplots
import pprint

In [2]:
data = pd.read_csv("../datasets/cardiac_data_cleaned_engineered.csv")

In [3]:
data.shape

(2812, 23)

In [8]:
#### to dispaly all feature without wrap in data.head()
pd.set_option('display.max_columns', None)

In [7]:
data.head()

,Facility_ID,Hospital_Name,Detailed_Region,Region,Procedure,Year_of_Hospital_Discharge,Number_of_Cases,Number_of_Deaths,Observed_Mortality_Rate,Expected_Mortality_Rate,Risk_Adjusted_Mortality_Rate,Lower_Limit_of_Confidence_Interval,Upper_Limit_of_Confidence_Interval,Comparison_Results,Start_Year,End_Year,Mid_Year,Comparison_Results_Category,Is_Higher_Than_Expected_Mortality,Is_Lower_Than_Expected_Mortality,Is_As_Expected_Mortality,Observed_vs_Expected_Difference,Observed_vs_RiskAdjusted_Difference
0,1,Albany Med. Ctr,Capital District,Capital District,All PCI,2016,680,17,2.50,1.52,2.03,1.18,3.26,Rate not different than Statewide Rate,2016,2016,2016,Rate not different than Statewide Rate,False,False,True,0.98,0.47
1,1045,White Plains Hospital,NY Metro - New Rochelle,NY Metro - New Rochelle,Non-Emergency PCI,2015,338,1,0.30,0.64,0.34,0.00,1.91,Rate not different than Statewide Rate,2015,2015,2015,Rate not different than Statewide Rate,False,False,True,-0.34,-0.04
2,1438,Bellevue Hospital Ctr,Manhattan,NY Metro - NYC,All PCI,2010,448,4,0.89,0.89,0.84,0.23,2.16,Rate not different than Statewide Rate,2010,2010,2010,Rate not different than Statewide Rate,False,False,True,0.00,0.05
3,1439,Beth Israel Med Ctr,Manhattan,NY Metro - NYC,All PCI,2010,1762,11,0.62,0.70,0.75,0.38,1.35,Rate not different than Statewide Rate,2010,2010,2010,Rate not different than Statewide Rate,False,False,True,-0.08,-0.13
4,1178,Bronx-Lebanon-Cncourse,Bronx,NY Metro - NYC,All PCI,2010,65,4,6.15,2.41,2.15,0.58,5.50,Rate not different than Statewide Rate,2010,2010,2010,Rate not different than Statewide Rate,False,False,True,3.74,4.00


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2812 entries, 0 to 2811
Data columns (total 23 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Facility_ID                          2812 non-null   int64  
 1   Hospital_Name                        2812 non-null   object 
 2   Detailed_Region                      2812 non-null   object 
 3   Region                               2812 non-null   object 
 4   Procedure                            2812 non-null   object 
 5   Year_of_Hospital_Discharge           2812 non-null   object 
 6   Number_of_Cases                      2812 non-null   int64  
 7   Number_of_Deaths                     2812 non-null   int64  
 8   Observed_Mortality_Rate              2812 non-null   float64
 9   Expected_Mortality_Rate              2812 non-null   float64
 10  Risk_Adjusted_Mortality_Rate         2812 non-null   float64
 11  Lower_Limit_of_Confidence_Inte

In [11]:
data.describe()

,Facility_ID,Number_of_Cases,Number_of_Deaths,Observed_Mortality_Rate,Expected_Mortality_Rate,Risk_Adjusted_Mortality_Rate,Lower_Limit_of_Confidence_Interval,Upper_Limit_of_Confidence_Interval,Start_Year,End_Year,Mid_Year,Observed_vs_Expected_Difference,Observed_vs_RiskAdjusted_Difference
count,2812.000000,2812.000000,2812.000000,2812.000000,2812.000000,2812.000000,2757.000000,2757.000000,2812.000000,2812.000000,2812.000000,2812.000000,2812.000000
mean,897.571124,1080.069701,18.603841,2.065509,1.950423,2.077287,0.900511,5.929300,2013.844595,2014.679587,2014.262091,0.115085,-0.011778
std,602.381608,4359.572902,74.354456,1.699924,1.202760,1.752055,0.967809,24.886388,2.991336,2.872502,2.890744,1.189586,0.734439
min,0.000000,1.000000,0.000000,0.000000,0.170000,0.000000,0.000000,0.580000,2008.000000,2010.000000,2009.000000,-7.450000,-6.290000
25%,456.500000,202.000000,3.000000,0.810000,0.947500,0.840000,0.170000,2.300000,2011.000000,2012.000000,2012.000000,-0.460000,-0.210000
50%,885.000000,375.000000,7.000000,1.605000,1.610000,1.560000,0.540000,3.980000,2014.000000,2015.000000,2014.000000,0.000000,0.000000
75%,1438.000000,702.250000,13.000000,2.992500,2.810000,3.000000,1.370000,6.000000,2016.000000,2017.000000,2017.000000,0.500000,0.210000
max,3058.000000,54276.000000,1021.000000,20.000000,8.060000,14.590000,6.660000,1228.420000,2019.000000,2019.000000,2019.000000,14.610000,12.620000


In [ ]:
def Columns_Info(data):
    """
    Display a table showing the number of unique values for each column in the DataFrame.
    """
    unique_counts = {
        "Column": data.columns,
        "Unique Values": [data[col].nunique() for col in data.columns],
        'null values': [data[col].isna().sum() for col in data.columns]
    }
    
    result_df = pd.DataFrame(unique_counts)
    display(result_df)
Columns_Info(data)

,Column,Unique Values,null values
0,Facility_ID,72,0
1,Hospital_Name,114,0
2,Detailed_Region,12,0
3,Region,8,0
4,Procedure,6,0
5,Year_of_Hospital_Discharge,20,0
6,Number_of_Cases,1219,0
7,Number_of_Deaths,116,0
8,Observed_Mortality_Rate,568,0
9,Expected_Mortality_Rate,474,0


## All Unique Values of Categorical Feature 

In [12]:
data['Region'].unique()

array(['Capital District', 'NY Metro - New Rochelle', 'NY Metro - NYC',
       'Western NY - Rochester', 'Western NY - Buffalo', 'Central NY',
       'NY Metro - Long Island', 'New York State'], dtype=object)

In [ ]:
data['Detailed_Region'].unique()    ## Region and detailed region has not sigmificant difference so we will remove this feature.

array(['Capital District', 'NY Metro - New Rochelle', 'Manhattan',
       'Bronx', 'Kings', 'Western NY - Rochester', 'Queens',
       'Western NY - Buffalo', 'Central NY', 'NY Metro - Long Island',
       'Staten Island', 'New York State'], dtype=object)

In [13]:
data['Procedure'].unique()

array(['All PCI', 'Non-Emergency PCI', 'Valve or Valve/CABG', 'TAVR',
       'CABG', 'Emergency PCI'], dtype=object)

In [ ]:
data['Comparison_Results'].unique()  ## Comparison_Results_Category also same as this feature

array(['Rate not different than Statewide Rate',
       'Rate significantly higher than Statewide Rate',
       'Rate significantly lower than Statewide Rate', 'Statewide Rate'],
      dtype=object)

In [20]:
data['Hospital_Name'].value_counts().head(15)

Hospital_Name
Mount Sinai Hospital       55
Lenox Hill Hospital        55
Maimonides Medical Ctr     55
Montefiore - Moses         55
North Shore Univ Hosp      55
Westchester Med Ctr        55
Montefiore - Weiler        55
Strong Memorial Hosp       55
St. Francis Hospital       55
Rochester General Hosp     55
Univ. Hosp-Stony Brook     55
NYS - All Hospitals        55
Vassar Bros. Med Ctr       54
St. Elizabeth Med Ctr      54
Staten Island Univ Hosp    53
Name: count, dtype: int64

In [ ]:
data['Year_of_Hospital_Discharge'].unique()  ### data collection year 

array(['2016', '2015', '2010', '2013-2015', '2008-2010', '2011',
       '2014-2016', '2009-2011', '2010-2012', '2012', '2014', '2012-2014',
       '2011-2013', '2013', '2017', '2015-2017', '2018', '2016-2018',
       '2019', '2017-2019'], dtype=object)

In [29]:
data['Is_As_Expected_Mortality'].unique()

array([ True, False])

In [ ]:
data['Is_Higher_Than_Expected_Mortality'].unique()  ## highre and lower fence has only false value, it does't make sence so we will simply remove this features.

array([False])

In [ ]:
data['Is_Lower_Than_Expected_Mortality'].unique()

array([False])

In [54]:

data['Is_As_Expected_Mortality'].unique()

array([ True, False])

In [ ]:
### we drop this features, this is not improtant and has unwanted values.

data.drop((['Is_Higher_Than_Expected_Mortality','Is_Lower_Than_Expected_Mortality','Detailed_Region','Facility_ID']) , axis=1, inplace=True)

In [162]:
fig = px.histogram(data, x="Lower_Limit_of_Confidence_Interval")
fig.show()

In [163]:
fig = px.histogram(data, x="Upper_Limit_of_Confidence_Interval")
fig.show()

In [144]:
fig = px.box(data, y="Lower_Limit_of_Confidence_Interval", points='all')
fig.update_layout(width = 1300, height = 600)
fig.show()

In [145]:
fig = px.box(data, y="Upper_Limit_of_Confidence_Interval", points='all')
fig.update_layout(width = 1300, height = 600)
fig.show()

In [ ]:
# upperlimit = data[data['Upper_Limit_of_Confidence_Interval'] ==  1228.42, ['Upper_Limit_of_Confidence_Interval'] = 204.32]
data.loc[data['Upper_Limit_of_Confidence_Interval'] == 1228.42, 'Upper_Limit_of_Confidence_Interval'] = 204.32



In [149]:
upperlimit['Upper_Limit_of_Confidence_Interval'].mean()


np.float64(4.004277821625888)

In [146]:
data['Upper_Limit_of_Confidence_Interval'].describe()

count    2757.000000
mean        5.929300
std        24.886388
min         0.580000
25%         2.300000
50%         3.980000
75%         6.000000
max      1228.420000
Name: Upper_Limit_of_Confidence_Interval, dtype: float64

In [135]:
data.head()


,Hospital_Name,Region,Procedure,Year_of_Hospital_Discharge,Number_of_Cases,Number_of_Deaths,Observed_Mortality_Rate,Expected_Mortality_Rate,Risk_Adjusted_Mortality_Rate,Lower_Limit_of_Confidence_Interval,Upper_Limit_of_Confidence_Interval,Comparison_Results,Start_Year,End_Year,Mid_Year,Comparison_Results_Category,Is_As_Expected_Mortality,Observed_vs_Expected_Difference,Observed_vs_RiskAdjusted_Difference
0,Albany Med. Ctr,Capital District,All PCI,2016,680,17,2.50,1.52,2.03,1.18,3.26,Rate not different than Statewide Rate,2016,2016,2016,Rate not different than Statewide Rate,True,0.98,0.47
1,White Plains Hospital,NY Metro - New Rochelle,Non-Emergency PCI,2015,338,1,0.30,0.64,0.34,0.00,1.91,Rate not different than Statewide Rate,2015,2015,2015,Rate not different than Statewide Rate,True,-0.34,-0.04
2,Bellevue Hospital Ctr,NY Metro - NYC,All PCI,2010,448,4,0.89,0.89,0.84,0.23,2.16,Rate not different than Statewide Rate,2010,2010,2010,Rate not different than Statewide Rate,True,0.00,0.05
3,Beth Israel Med Ctr,NY Metro - NYC,All PCI,2010,1762,11,0.62,0.70,0.75,0.38,1.35,Rate not different than Statewide Rate,2010,2010,2010,Rate not different than Statewide Rate,True,-0.08,-0.13
4,Bronx-Lebanon-Cncourse,NY Metro - NYC,All PCI,2010,65,4,6.15,2.41,2.15,0.58,5.50,Rate not different than Statewide Rate,2010,2010,2010,Rate not different than Statewide Rate,True,3.74,4.00


# Analysis

In [48]:
avg_mortality_rate = data['Observed_Mortality_Rate'].mean()
print(f"Average Mortality rate : {avg_mortality_rate:.2f}%")

Average Mortality rate : 2.07%


In [49]:
Expected_mortality_rate = data['Expected_Mortality_Rate'].mean()
print(f"Average Expected Mortality rate : {Expected_mortality_rate:.2f}%")

Average Expected Mortality rate : 1.95%


In [50]:
risk_adjusted_mortality_rate = data['Risk_Adjusted_Mortality_Rate'].mean()
print(f"Average Risk Adjusted Mortality rate : {risk_adjusted_mortality_rate:.2f}%")

Average Risk Adjusted Mortality rate : 2.08%


In [51]:
obseved_vs_expected = data['Observed_vs_Expected_Difference'].mean()
print(f"Observed vs Expected  Mortality Difference Average : {obseved_vs_expected:.2f}%")

Observed vs Expected  Mortality Difference Average : 0.12%


In [59]:
value_counts = data['Is_As_Expected_Mortality'].value_counts().reset_index()
value_counts.columns = ['Is_As_Expected_Mortality', 'Count']

fig = px.pie(
    value_counts,
    names='Is_As_Expected_Mortality',
    values='Count',
    title='Distribution of True vs False',
    color='Is_As_Expected_Mortality',
    color_discrete_map={True: 'green', False: 'red'}
)
fig.update_layout(
    width = 600
    
)

fig.show()


### Here we can see that above 90 % expected mortality is True Comparatively Observed mortality.

In [68]:
data['Start_Year'].max()

np.int64(2019)

In [90]:
group_observed = data.groupby('Start_Year', as_index=False)['Observed_Mortality_Rate'].mean()
group_expected = data.groupby('Start_Year', as_index=False)['Expected_Mortality_Rate'].mean()
group_rist_adjusted = data.groupby('Start_Year', as_index=False)['Risk_Adjusted_Mortality_Rate'].mean()

fig = go.Figure()

fig.add_trace(go.Scatter(
    y=group_observed['Observed_Mortality_Rate'],
    x=group_observed['Start_Year'],
    name='Observed Mortality Rate',
    
    marker=dict(size=6)
))

fig.add_trace(go.Scatter(
    y=group_expected['Expected_Mortality_Rate'],
    x=group_expected['Start_Year'],
    name='Expected Mortality Rate',
    line = dict(color='grey', width=4, dash='dot'),
    marker=dict(size=6)
))

fig.add_trace(go.Scatter(
    y=group_rist_adjusted['Risk_Adjusted_Mortality_Rate'],
    x=group_rist_adjusted['Start_Year'],
    line=dict(color='#FF4500', width=4, dash='dot'),
    name='Risk Adjusted Mortality Rate',
    marker=dict(size=6)
))

# Step 3: Update layout
fig.update_layout(
    title=dict(
        text='Observed Mortality Rate (2008-2019)',
        font=dict(size=22, color='saddlebrown'),
        x=0.5
    ),
    xaxis_title='Year',
    yaxis_title='Observed Mortality Rate',
    template='plotly_white',
    height=500,
    width=900
)

fig.show()

### 🔑 Key Points
* Average mortality rates are decreasing over time.

* Environmental pollution and lifestyle-related diseases are on the rise.

* People are becoming more vulnerable to chronic illnesses.

* Medical advancements and better healthcare systems are helping reduce death rates.

* The decline in mortality does not necessarily reflect better overall health.

In [107]:
group_observed = data.groupby('Start_Year', as_index=False)['Number_of_Cases'].mean()
group_expected = data.groupby('Start_Year', as_index=False)['Number_of_Deaths'].mean()

fig = go.Figure()

fig.add_trace(go.Scatter(
    y=group_observed['Number_of_Cases'],
    x=group_observed['Start_Year'],
    name='Average No. of cases',
    line = dict(color='green', width=4, dash='dot'),
    marker=dict(size=6)
))

fig.add_trace(go.Scatter(
    y=group_expected['Number_of_Deaths'],
    x=group_expected['Start_Year'],
    line=dict(color='red', width=4, dash='dot'),
    name='Average No. deaths',
    marker=dict(size=6)
))

fig.update_layout(
    title=dict(
        text='NO. of Cases and Death per year ',
        font=dict(size=22, color='saddlebrown'),
        x=0.5
    ),
    xaxis_title='Year',
    yaxis_title='No. of cases',
    template='plotly_white',
    height=500,
    width=900
)

fig.show()

### 🔍 Observations & Insights:

* The average number of cases is gradually increasing over the years.

* Despite more cases, the number of deaths remains relatively constant, showing no significant upward trend.

* Interestingly, the average deaths in 2008 were higher than in 2018 and 2019.

* This indicates a significant improvement in treatment quality and healthcare performance over time.

* Better clinical outcomes, even with more patient load, suggest enhanced hospital efficiency and capability.

In [120]:
mortality_difference= data.groupby('Start_Year', as_index=False)['Observed_vs_Expected_Difference'].mean().round(2)

fig = px.line(mortality_difference, x="Start_Year", y="Observed_vs_Expected_Difference",markers=True)

fig.update_layout(
    title=dict(
        text='Observed vs Expected Difference Trends',
        font=dict(size=22, color='black'),
        x=0.5
    ),
    xaxis_title='Year',
    yaxis_title='Difference (%)',
    template='plotly_white',
    height=500,
    width=900
)
fig.show()

In [134]:
procedure_counts = data['Procedure'].value_counts().reset_index()
procedure_counts.columns = ['Procedure', 'Count']

 
blue_shades = ['#0059b3', '#1a75ff', '#4da6ff', '#80c1ff', '#b3daff', '#e6f2ff'] ## custom colors 

fig = px.pie(
    procedure_counts,
    names='Procedure',
    values='Count',
    title='Distribution of Procedure',
    # color='Procedure',
    color_discrete_map={True: 'green', False: 'red'},
    color_discrete_sequence= blue_shades
)
fig.update_layout(
    width = 700
    
)

fig.show()


In [ ]:
difference_of_confidence_interval = data['Upper_Limit_of_Confidence_Interval'] - data['Lower_Limit_of_Confidence_Interval']
difference_of_confidence_interval

0       2.08
1       1.91
2       1.93
3       0.97
4       4.92
        ... 
2807     NaN
2808     NaN
2809     NaN
2810     NaN
2811     NaN
Length: 2812, dtype: float64

In [ ]:
data['CI_Difference'] = data['Upper_Limit_of_Confidence_Interval'] - data['Lower_Limit_of_Confidence_Interval']  

ci_diff_by_year = data.groupby('Start_Year', as_index=False)['CI_Difference'].mean().round(2) 

fig = px.line(
    ci_diff_by_year,
    x="Start_Year",
    y="CI_Difference",
    markers=True,
    title='Year-wise Confidence Interval Difference Trend'
)

fig.update_layout(
    title=dict(
        text='Year-wise Confidence Interval Difference Trend',
        font=dict(size=22, color='black'),
        x=0.5
    ),
    xaxis_title='Year',
    yaxis_title='CI Width (Upper - Lower)',
    template='plotly_white',
    height=500,
    width=900,
    xaxis=dict(
        tickmode='linear',
        dtick=1
    )
)

fig.show()


In [160]:
group_observed = data.groupby('Start_Year', as_index=False)['Upper_Limit_of_Confidence_Interval'].mean()
group_expected = data.groupby('Start_Year', as_index=False)['Lower_Limit_of_Confidence_Interval'].mean()

fig = go.Figure()

fig.add_trace(go.Scatter(
    y=group_observed['Upper_Limit_of_Confidence_Interval'],
    x=group_observed['Start_Year'],
    name='Upper_Limit_of_Confidence_Interval',
    line = dict(color='green', width=4, dash='dot'),
    marker=dict(size=6)
))

fig.add_trace(go.Scatter(
    y=group_expected['Lower_Limit_of_Confidence_Interval'],
    x=group_expected['Start_Year'],
    line=dict(color='red', width=4, dash='dot'),
    name='Lower_Limit_of_Confidence_Interval',
    marker=dict(size=6)
))

fig.update_layout(
    title=dict(
        text='Year-wise Confidence Interval Difference Trend ',
        font=dict(size=22, color='saddlebrown'),
        x=0.5
    ),
    xaxis_title='Year',
    yaxis_title='No. of cases',
    template='plotly_white',
    height=500,
    width=1300
)

fig.show()

In [ ]:
upperlimit['Upper_Limit_of_Confidence_Interval'].mean()


np.float64(4.004277821625888)

In [61]:
data.head()

,Hospital_Name,Region,Procedure,Year_of_Hospital_Discharge,Number_of_Cases,Number_of_Deaths,Observed_Mortality_Rate,Expected_Mortality_Rate,Risk_Adjusted_Mortality_Rate,Lower_Limit_of_Confidence_Interval,Upper_Limit_of_Confidence_Interval,Comparison_Results,Start_Year,End_Year,Mid_Year,Comparison_Results_Category,Is_As_Expected_Mortality,Observed_vs_Expected_Difference,Observed_vs_RiskAdjusted_Difference
0,Albany Med. Ctr,Capital District,All PCI,2016,680,17,2.50,1.52,2.03,1.18,3.26,Rate not different than Statewide Rate,2016,2016,2016,Rate not different than Statewide Rate,True,0.98,0.47
1,White Plains Hospital,NY Metro - New Rochelle,Non-Emergency PCI,2015,338,1,0.30,0.64,0.34,0.00,1.91,Rate not different than Statewide Rate,2015,2015,2015,Rate not different than Statewide Rate,True,-0.34,-0.04
2,Bellevue Hospital Ctr,NY Metro - NYC,All PCI,2010,448,4,0.89,0.89,0.84,0.23,2.16,Rate not different than Statewide Rate,2010,2010,2010,Rate not different than Statewide Rate,True,0.00,0.05
3,Beth Israel Med Ctr,NY Metro - NYC,All PCI,2010,1762,11,0.62,0.70,0.75,0.38,1.35,Rate not different than Statewide Rate,2010,2010,2010,Rate not different than Statewide Rate,True,-0.08,-0.13
4,Bronx-Lebanon-Cncourse,NY Metro - NYC,All PCI,2010,65,4,6.15,2.41,2.15,0.58,5.50,Rate not different than Statewide Rate,2010,2010,2010,Rate not different than Statewide Rate,True,3.74,4.00
